In [ ]:
import ee
import geemap
import os
import random
ee.Authenticate()
ee.Initialize(project = 'tony-67130749')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


In [ ]:
import datetime

CLEAR_THRESHOLD = 0.80
QA_BAND = 'cs_cdf'
vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 64, 'max': 5454, 'gamma': 1.8}

# Define ROI (replace with your actual ROI)
# ROI = ee.Geometry.Polygon([[[102.0, 15.0], [102.5, 15.0], [102.5, 15.5], [102.0, 15.5], [102.0, 15.0]]])

countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017");
country_name = "Cambodia";

ROI = countries.filter(ee.Filter.eq("country_na", country_name));


# Correcting the centroid function by specifying a non-zero error margin
def download_images(start_date, end_date, interval_days):
    # Convert input dates to EE format
    start = ee.Date(start_date)
    end = ee.Date(end_date)

    # Calculate the total number of intervals
    diff = end.difference(start, 'day').divide(interval_days).ceil().getInfo()

    # Loop through each interval
    for i in range(int(diff)):
        interval_start = start.advance(i * interval_days, 'day')
        interval_end = interval_start.advance(interval_days, 'day')

        # Load Sentinel-2 image collection
        s2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")\
            .filterBounds(ROI)\
            .filterDate(interval_start, interval_end)\
            .sort("CLOUDY_PIXEL_PERCENTAGE")

        # Load Cloud Score Plus collection
        cs_plus = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')\
            .filterBounds(ROI)\
            .filterDate(interval_start, interval_end)

        # Link the collections and apply the mask
        def add_cs_band(img):
            cs_image = cs_plus.filter(ee.Filter.eq('system:index', img.get('system:index'))).first()
            return img.addBands(cs_image.select(QA_BAND))

        image_collection = s2.map(add_cs_band)\
                             .map(mask_data)\
                             .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12'])

        # Get the median image to reduce the collection
        median_image = image_collection.median().clip(ROI)

        # Create a binary image and clip it to the ROI
        binary_image = median_image.unmask(0, False).clip(ROI).eq(0)

        # Mask the edges to exclude boundary polygons
        no_boundary_image = binary_image.updateMask(binary_image.clip(ROI))

        # Convert the binary image to polygons
        polygons = no_boundary_image.reduceToVectors(
            geometryType='polygon',
            reducer=ee.Reducer.mean(),  # Correct usage of the reducer
            scale=30,
            geometry=ROI,  # Use the region of interest geometry
            maxPixels=1e13,
            labelProperty='label',
            bestEffort=True  # Simplifies the result if necessary
        )

        # Create grid for sampling
        grid = polygons.geometry().coveringGrid('EPSG:4326', 5000)

        # Specify a small error margin when calculating the centroid
        def centroid_with_error_margin(feat):
            return feat.centroid(maxError=1)  # Use a non-zero error margin

        sample_grid = grid.map(centroid_with_error_margin)

        # Print layers to console for visualization (if needed)
        print(f"Sentinel-2 Image Interval {interval_start.format('YYYY-MM-dd').getInfo()} to {interval_end.format('YYYY-MM-dd').getInfo()}")

        # Export the sample grid as an asset for each interval
        task = ee.batch.Export.table.toAsset(
            collection=sample_grid,
            description=f'sample_of_missing_image_{interval_start.format("YYYY-MM-dd").getInfo()}',
            assetId=f'projects/tony-67130749/assets/sample_of_missing_image_{interval_start.format("YYYY-MM-dd").getInfo()}'
        )
        task.start()


In [ ]:
# Call the function to download images
download_images('2023-01-01', '2023-01-31', 10)

Sentinel-2 Image Interval 2023-01-01 to 2023-01-11
Sentinel-2 Image Interval 2023-01-11 to 2023-01-21
Sentinel-2 Image Interval 2023-01-21 to 2023-01-31
